In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import IPython.display as ipd
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
from pathlib import Path
import numpy as np
import json
import torch
from pprint import pprint

from sins.database.database import SINS, AudioReader
from sins.database.utils import prepare_sessions
from sins.systems.sad.model import BinomialClassifier
from sins.systems.modules import CNN2d, CNN1d, AutoPool
from sins.features.stft import STFT
from sins.features.mel_transform import MelTransform
from sins.features.normalize import Normalizer
from sins.systems.utils import Collate
from sins import paths
from sins.systems.sad.utils import load_sections, get_sections_in_range


In [ ]:
exp_dir = paths.exp_dir / 'sad' / '2019-10-07-06-57-23'
with (exp_dir / '1' / 'config.json').open() as f:
    config = json.load(f)

## Load Model

In [ ]:
model = BinomialClassifier(
    cnn_2d=CNN2d(**config['model']['cnn_2d']),
    cnn_1d=CNN1d(**config['model']['cnn_1d']),
    pooling=AutoPool(**config['model']['pool'])
)
ckpt = torch.load(exp_dir / 'ckpt-best.pth')
model.load_state_dict(ckpt)
model.eval()
model.pooling.alpha = 2.0

pool_sizes = [
    pool_size[1] if isinstance(pool_size, (list, tuple)) else pool_size
    for pool_size in (model._cnn_2d.pool_sizes + model._cnn_1d.pool_sizes)
]
total_pool_size = np.prod(pool_sizes)

## Example sad

In [ ]:
db = SINS()
presence = prepare_sessions(
    db.sessions, room='living', include_absence=True,
    discard_other_rooms=True, discard_ambiguities=False,
    label_map_fn=lambda label: (False if label == "absence" else True)
)
eval_sets = db.get_segments(
    db.room_to_nodes['living'], max_segment_length=60., time_ranges=db.eval_ranges, sessions=presence,
    session_key='presence'
)

audio_reader = AudioReader(**config['audio_reader'])
stft = STFT(**config['stft'])
mel_transform = MelTransform(**config['mel_transform'])
normalizers = [
    Normalizer("mel_transform", name=node, **config['normalizer']) for node in db.room_to_nodes['living']
]
[normalizer.initialize_moments(verbose=True) for normalizer in normalizers]

def pretend_batch(example):
    example['features'] = torch.Tensor(example['mel_transform'].transpose((0,2,1))[:, None])
    example['seq_len'] = 4*[example['mel_transform'].shape[-1]]
    return example

eval_sets = [
    ds.map(audio_reader).map(stft).map(mel_transform).map(normalizer).map(pretend_batch)
    for ds, normalizer in zip(eval_sets, normalizers)
]

In [ ]:
example = eval_sets[0][200]
print(example['presence'])

In [ ]:
sad, seq_len = model(example)
sad = sad.cpu().data.numpy().mean((0,1))

In [ ]:
n_on, n_off = 0, 300
x = example['mel_transform'][0, 10*n_on:10*n_off].T
y = sad[n_on:n_off]
fig, axes = plt.subplots(2,1)
axes[0].imshow(x, interpolation='nearest', aspect='auto', origin="lower")
axes[1].plot(y, linewidth=2)
axes[1].set_xlim([-0.5, y.shape[0]-0.5])

## Evaluate sad

In [ ]:
from sins.systems.sad.evaluate import prepare_dataset, fscore, prepare_targets, simple_sad

In [ ]:
sound_events_dir = '/path/to/dcase2016_task2_train_sounds'

### test metric

In [ ]:
scores = np.array([[[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]]])
targets = prepare_targets(
    scores, onset_frames=np.array([6]), offset_frames=np.array([12.])
)
print(targets)
print(fscore(targets, scores>0.5))

### plot example mix

In [ ]:
eval_sets, onset_frames, offset_frames = prepare_dataset(
    exp_dir, sound_events_dir, mixtures_per_sound=1, segment_length=20., snr=0.,
    nodes=["Node1"], num_workers=8, prefetch_buffer=16, seed=0, dataset_name='eval',
    prefetch=False
)
example_idx = 100
batch = eval_sets[0][example_idx]
y, seq_len = model(Collate()(batch))
y = y.cpu().data.numpy().mean((0,1))
x = batch[1]['features'][0]
targets = prepare_targets(
    y[None,None], 
    np.array([onset_frames[example_idx] / total_pool_size]),
    np.array([offset_frames[example_idx] / total_pool_size])
)[0, 0]
print(targets.shape)

n_on = 0 
n_off = 1000
x = x[:, n_on:n_off]
y = y[n_on//total_pool_size:n_off//total_pool_size]
targets = targets[n_on//total_pool_size:n_off//total_pool_size]

fig, axes = plt.subplots(3,1)
axes[0].imshow(
    x,#[...,int(onset_frames[n]-10):int(offset_frames[n]+10)],
    interpolation='nearest', aspect='auto', origin="lower"
)
axes[1].plot(targets, linewidth=2)
axes[1].set_ylim([0., 1.])
axes[1].set_xlim([-0.5, targets.shape[-1]-0.5])
axes[2].plot(y, linewidth=2)
axes[2].set_ylim([0., 1.])
axes[2].set_xlim([-0.5, y.shape[-1]-0.5]);

## Listen to active sections

In [ ]:
nodes = config["nodes"]
sections = load_sections(exp_dir / 'ensemble_sections.json')
train_sections = get_sections_in_range(sections, db.train_ranges)
datasets = db.get_segments(
    nodes, min_segment_length=1., max_segment_length=60., 
    time_ranges=train_sections
)
datasets = [ds.map(audio_reader) for ds in datasets]

In [ ]:
sec_idx = 100
for ds in datasets:
    ipd.display(ipd.Audio(ds[sec_idx]['audio_data'][0], rate=16000))

## Correlate Node sections

In [ ]:
def compute_overlap(sections_a, sections_b, shift=0.):
    sections_b = [(start - shift, stop - shift) for (start, stop) in sections_b]
    overlap = 0.
    idx_b = 0
    for section_a in sections_a:
        while idx_b < len(sections_b) and sections_b[idx_b][-1] < section_a[-2]:
            idx_b += 1
        while idx_b < len(sections_b) and sections_b[idx_b][-1] < section_a[-1]:
            overlap += sections_b[idx_b][-1] - max(section_a[-2], sections_b[idx_b][-2])
            idx_b += 1
        if idx_b < len(sections_b):
            overlap += max(
                section_a[-1] - max(section_a[-2], sections_b[idx_b][-2]), 0.)
    return overlap

def correlate_segments(sections_a, sections_b, shifts=np.arange(-1., 1.1, .2)):
    return np.array([
        compute_overlap(sections_a, sections_b, shift) for shift in shifts
    ])

def compute_shift_mat(sections, shifts=np.arange(-1., 1.1, .2)):
    shift_mat = np.array([
        [
            shifts[np.argmax(correlate_segments(
                sections[node_1], sections[node_2], shifts
            ))] for node_2 in range(len(sections))
        ]
        for node_1 in range(len(sections))
    ])
    return shift_mat

In [ ]:
node_sections = []
for node in db.room_to_nodes['living']:
    with (exp_dir / f'{node}_sections.json').open() as f:
        node_sections.append(json.load(f))
shifts = np.arange(-1., 1.1, .1)
autocorrelation = correlate_segments(node_sections[5], node_sections[5], shifts=shifts)
crosscorrelation = correlate_segments(node_sections[5], node_sections[6], shifts=shifts)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,2))
ax.plot(shifts, autocorrelation/1000)
ax.plot(shifts, crosscorrelation/1000)
ax.set_xlim(-1,1)
ax.set_xlabel('Lag / s', size=14)
ax.set_ylabel('Correlation / ($10^3$ s)', size=14)
ax.legend(['Auto', 'Cross'], fontsize=13)
ax.tick_params(axis='both', which='major', labelsize=13)
ax.grid()
plt.locator_params(axis='x', nbins=5)
plt.savefig('correlation.pdf', bbox_inches = 'tight', pad_inches = 0.02)

In [ ]:
shift_mat = compute_shift_mat(node_sections)
shifts = shift_mat[0]
shift_mat = shift_mat - shifts + shifts[:, None]
assert (np.abs(shift_mat) < 0.1).all()